In [ ]:
#submission 2 - xgboost

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
%matplotlib inline

from pathlib import Path

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split

In [ ]:
train_values = pd.read_csv('/content/gdrive/MyDrive/tp2/train_values.csv', index_col='building_id')
train_labels = pd.read_csv('/content/gdrive/MyDrive/tp2/train_labels.csv', index_col='building_id')

In [ ]:
test_values = pd.read_csv('/content/gdrive/MyDrive/tp2/test_values.csv', index_col='building_id')

In [ ]:
train_values.dtypes

geo_level_1_id                             int64
geo_level_2_id                             int64
geo_level_3_id                             int64
count_floors_pre_eq                        int64
age                                        int64
area_percentage                            int64
height_percentage                          int64
land_surface_condition                    object
foundation_type                           object
roof_type                                 object
ground_floor_type                         object
other_floor_type                          object
position                                  object
plan_configuration                        object
has_superstructure_adobe_mud               int64
has_superstructure_mud_mortar_stone        int64
has_superstructure_stone_flag              int64
has_superstructure_cement_mortar_stone     int64
has_superstructure_mud_mortar_brick        int64
has_superstructure_cement_mortar_brick     int64
has_superstructure_t

In [ ]:
columnas = ['land_surface_condition',
            'foundation_type',
            'roof_type',
            'ground_floor_type',
            'other_floor_type',
            'position',
            'plan_configuration',
            'legal_ownership_status',
            'geo_level_1_id',
            'geo_level_2_id',
            'geo_level_3_id'];

In [ ]:
for row in columnas:
  # Calculamos la frecuencia por cada grado de cada valor de la variable
  Auxiliar = test_values.groupby([row]).size()/len(test_values);

  # Reemplazamos la columna por la columna de frecuencias
  test_values.loc[:,row] = test_values[row].map(Auxiliar);

In [ ]:
for row in columnas:
  # Calculamos la frecuencia por cada grado de cada valor de la variable
  Auxiliar = train_values.groupby([row]).size()/len(train_values);

  # Reemplazamos la columna por la columna de frecuencias
  train_values.loc[:,row] = train_values[row].map(Auxiliar);

In [ ]:
train_values.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 260601 entries, 802906 to 747594
Data columns (total 38 columns):
 #   Column                                  Non-Null Count   Dtype  
---  ------                                  --------------   -----  
 0   geo_level_1_id                          260601 non-null  float64
 1   geo_level_2_id                          260601 non-null  float64
 2   geo_level_3_id                          260601 non-null  float64
 3   count_floors_pre_eq                     260601 non-null  int64  
 4   age                                     260601 non-null  int64  
 5   area_percentage                         260601 non-null  int64  
 6   height_percentage                       260601 non-null  int64  
 7   land_surface_condition                  260601 non-null  float64
 8   foundation_type                         260601 non-null  float64
 9   roof_type                               260601 non-null  float64
 10  ground_floor_type                      

In [ ]:
selected_features = ['land_surface_condition',
            'foundation_type',
            'roof_type',
            'ground_floor_type',
            'other_floor_type',
            'position',
            'plan_configuration',
            'legal_ownership_status',
            'geo_level_1_id',
            'geo_level_2_id',
            'geo_level_3_id'];
train_values_subset = train_values[selected_features]

In [ ]:
train_values_subset

,land_surface_condition,foundation_type,roof_type,ground_floor_type,other_floor_type,position,plan_configuration,legal_ownership_status,geo_level_1_id,geo_level_2_id,geo_level_3_id
building_id,,,,,,,,,,,
802906,0.831758,0.841117,0.701617,0.804368,0.634234,0.164604,0.959597,0.962924,0.093557,0.001036,0.000142
28830,0.031911,0.841117,0.701617,0.095460,0.634234,0.775477,0.959597,0.962924,0.073215,0.000764,0.000061
94947,0.831758,0.841117,0.701617,0.804368,0.166722,0.164604,0.959597,0.962924,0.057133,0.006754,0.000522
590882,0.831758,0.841117,0.701617,0.804368,0.166722,0.775477,0.959597,0.962924,0.023991,0.000787,0.000119
201944,0.831758,0.841117,0.701617,0.804368,0.166722,0.775477,0.959597,0.962924,0.031542,0.003983,0.000468
...,...,...,...,...,...,...,...,...,...,...,...
688636,0.136331,0.841117,0.701617,0.804368,0.152889,0.775477,0.021842,0.962924,0.021581,0.000445,0.000054
669485,0.831758,0.841117,0.701617,0.804368,0.634234,0.775477,0.959597,0.962924,0.083703,0.001002,0.000188
602512,0.831758,0.841117,0.236285,0.804368,0.634234,0.775477,0.959597,0.962924,0.083703,0.001278,0.000084


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train_values_subset,train_labels,test_size = 0.2);

In [ ]:
#modelos

In [ ]:
import xgboost as xgb

In [ ]:
# for preprocessing the data
from sklearn.preprocessing import StandardScaler

# the model
from sklearn.ensemble import RandomForestClassifier

# for combining the preprocess with model training
from sklearn.pipeline import make_pipeline

# for optimizing the hyperparameters of the pipeline
from sklearn.model_selection import GridSearchCV

In [ ]:
pipe = make_pipeline(StandardScaler(), 
                     xgb.XGBClassifier(random_state=2018))
pipe

Pipeline(memory=None,
         steps=[('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('xgbclassifier',
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, gamma=0, learning_rate=0.1,
                               max_delta_step=0, max_depth=3,
                               min_child_weight=1, missing=None,
                               n_estimators=100, n_jobs=1, nthread=None,
                               objective='binary:logistic', random_state=2018,
                               reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
                               seed=None, silent=None, subsample=1,
                               verbosity=1))],
         verbose=False)

In [ ]:
param_grid = {'xgbclassifier__n_estimators': [50, 100],
              'xgbclassifier__max_depth': [3, 4]}

In [ ]:
gs = GridSearchCV(pipe, param_grid, cv=5)

In [ ]:
for param in gs.get_params().keys():
    print(param)

cv
error_score
estimator__memory
estimator__steps
estimator__verbose
estimator__standardscaler
estimator__xgbclassifier
estimator__standardscaler__copy
estimator__standardscaler__with_mean
estimator__standardscaler__with_std
estimator__xgbclassifier__base_score
estimator__xgbclassifier__booster
estimator__xgbclassifier__colsample_bylevel
estimator__xgbclassifier__colsample_bynode
estimator__xgbclassifier__colsample_bytree
estimator__xgbclassifier__gamma
estimator__xgbclassifier__learning_rate
estimator__xgbclassifier__max_delta_step
estimator__xgbclassifier__max_depth
estimator__xgbclassifier__min_child_weight
estimator__xgbclassifier__missing
estimator__xgbclassifier__n_estimators
estimator__xgbclassifier__n_jobs
estimator__xgbclassifier__nthread
estimator__xgbclassifier__objective
estimator__xgbclassifier__random_state
estimator__xgbclassifier__reg_alpha
estimator__xgbclassifier__reg_lambda
estimator__xgbclassifier__scale_pos_weight
estimator__xgbclassifier__seed
estimator__xgbclassi

In [ ]:
gs.fit(X_train, y_train.values)

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), 

GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('standardscaler',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('xgbclassifier',
                                        XGBClassifier(base_score=0.5,
                                                      booster='gbtree',
                                                      colsample_bylevel=1,
                                                      colsample_bynode=1,
                                                      colsample_bytree=1,
                                                      gamma=0,
                                                      learning_rate=0.1,
                                                      max_delta_step=0,
                      

In [ ]:
from sklearn.metrics import f1_score

In [ ]:
in_sample_preds = gs.predict(X_train)
f1_score(y_train, in_sample_preds, average='micro')

0.6857060629316961

In [ ]:
in_sample_preds = gs.predict(X_test)
f1_score(y_test, in_sample_preds, average='micro')

0.6850022064043284

In [ ]:
#test_values = pd.read_csv('/content/gdrive/MyDrive/tp2/test_values.csv', index_col='building_id')

In [ ]:
test_values_subset = test_values[selected_features]
#test_values_subset = pd.get_dummies(test_values_subset)

In [ ]:
predictions = gs.predict(test_values_subset)

In [ ]:
submission_format = pd.read_csv('/content/gdrive/MyDrive/tp2/submission_format.csv', index_col='building_id')

In [ ]:
my_submission = pd.DataFrame(data=predictions,
                             columns=submission_format.columns,
                             index=submission_format.index)

In [ ]:
my_submission.head()

,damage_grade
building_id,
300051,2
99355,2
890251,2
745817,2
421793,2


In [ ]:
my_submission.to_csv('/content/gdrive/MyDrive/tp2/submission2.csv')

In [ ]:
!head submission.csv

head: cannot open 'submission.csv' for reading: No such file or directory


In [ ]:
my_submission.damage_grade.value_counts()

2    65910
3    18035
1     2923
Name: damage_grade, dtype: int64